In [21]:
import pandas as pd
import numpy as np 
import scipy as sp
import statsmodels.formula.api as smf
from tickers_list import tickers
pd.set_option('display.max_rows', 500)

Here we are cleaning the overall stock data from factset. 
- The problem we are having is that factset's numbers are not in a floating point number format (AKA Decimal). So we had to srip out the commas in the data set to convert into a float number. 

In [12]:
stock_data = pd.read_csv("Factset_Screen_Data\stock_data.csv")
# Rename columns Symbol to Ticker
stock_data.rename(columns={'Symbol':'Ticker'}, inplace=True)
# Make Ticker the index
stock_data.set_index('Ticker', inplace=True)
# Remove commas in Recent Closing Price, Market Value, Purchase Price, Common Shares Columns
stock_data['Recent Closing Price'] = stock_data['Recent Closing Price'].str.replace(',', '').astype(float)
stock_data['Market Value'] = stock_data['Market Value'].str.replace(',', '').astype(float)
stock_data['Purchase Price'] = stock_data['Purchase Price'].str.replace(',', '').astype(float)
stock_data['Common Shares'] = stock_data['Common Shares'].str.replace(',', '').astype(float)
stock_data.head(5)

,Name,Stock Exchange,RBICS Economy,Recent Closing Price,Market Value,Purchase Price,Common Shares
Ticker,,,,,,,
A,"Agilent Technologies, Inc.",NYSE,Healthcare,130.6,38651.1,43.6,324.0
AA,Alcoa Corporation,NYSE,Non-Energy Materials,39.4,7085.5,28.1,182.9
AAL,"American Airlines Group, Inc.",NASDAQ,Industrials,13.2,8578.0,46.7,507.3
AAON,"AAON, Inc.",NASDAQ,Industrials,55.6,2950.8,33.1,52.7
AAP,"Advance Auto Parts, Inc.",NYSE,Consumer Cyclicals,169.9,10211.7,169.1,73.7


The problem we are having is to try to get time series data in Factset of the individual stocks since we wanted to apply modern portfolio theory to our allocation. Here are some problems we ran into. 
* Factset does not have time series data for a list of stocks. 
* Yfinance has a rate limiter which is extreamly slow. In perspective it would take over 30 mins to get all data for the ticker list each call!
* Quandl for NASDAQ is not working properly. 
* But Wharton WRDS was our best option without paying for an API key from Alpha Vantage, Morning Start, ect. 

In [5]:

df1 = pd.read_csv("Wrds_Data\wrds_data_raw.csv")
df2 = df1.pivot_table(index='MthCalDt', columns="Ticker", values="MthPrc", aggfunc='first')
remove_strings_df = df2.apply(lambda x: x.str.replace(',','')) # Brute Force way to remove strings. 
#remove_strings_df.to_csv("Wrds_Data\wrds_data_clean.csv")
remove_strings_df.head(5)

Ticker,A,AA,AAL,AAON,AAP,AAPL,AAT,AAWW,AB,ABBV,...,YUMC,YY,Z,ZBH,ZBRA,ZD,ZEN,ZION,ZTS,ZWS
MthCalDt,,,,,,,,,,,,,,,,,,,,,
20170131,48.97,36.45,44.25,33.95,164.24,121.35,42.93,52.75,23.35,61.11,...,27.48,41.08,35.38,118.33,83.67,83.81,23.93,42.19,54.94,22.09
20170228,51.3,34.59,46.36,33.65,156.61,136.99,44,56.85,23.7,61.84,...,26.59,44.29,33.94,117.08,90.71,81.42,27.23,44.9,53.31,22.17
20170331,52.87,34.4,42.3,35.35,148.26,143.66,41.84,55.45,22.85,65.16,...,27.2,46.11,33.67,122.11,91.25,83.91,28.04,42,53.37,23.08
20170428,55.05,33.73,42.62,36.65,142.14,143.65,42.83,58,22.9,65.94,...,34.12,48.97,39,119.65,94.27,90.24,28.75,40.03,56.11,24.4
20170531,60.34,32.94,48.41,36.175,133.63,152.76,39.05,48.7,22.55,66.02,...,38.41,58.34,43.52,119.21,104.34,84.62,25.98,40.07,62.28,22.8


Here we are converting the cleaned data from a string into a float. The problem that I was having is that the Data had commas in the numbers where pandas was reading it. '1,560.09'

* pt_ret is the arthematic returns of the stocks
* pt_log is the natural log returns of the stocks. (Harmonic)

In [6]:
cleaned_data = remove_strings_df.astype(float)
pt_ret = cleaned_data.pct_change(1)[1:]
pt_log = np.log(cleaned_data/cleaned_data.shift(1))[1:]

Here we are adding market data and damodarans ERP for the stocks since we want a forward looking return to continue to beat the S&P500
* The Risk Free Rate is the 10yr US Treasury Note 
* Equity Risk Premium is the recent damodarn ERP. 

In [46]:
risk_free_rate = 0.0420 # 10 year treasury rate as of 10/23/2022 at 7:09pm (From Bloomberg)
equity_risk_premium = 0.0530 # Damodaran's estimate for October 2022
arth_Sigma = pt_ret.cov() # Covariance Martrix of the arthmatic returns
log_Sigma = pt_log.cov() # Covariance Matrix of the log returns


arth_pt_var = pt_ret.var()*12 # Variance of the arthmatic returns
log_pt_var = pt_log.var()*12 # Variance of the log returns


arth_pt_dvar = pt_ret[pt_ret < 0].var()*12 # Downside Variance of the arthmatic returns
log_pt_dvar = pt_log[pt_log < 0].var()*12 # Downside Variance of the log returns



log_Sigma

Ticker,A,AA,AAL,AAON,AAP,AAPL,AAT,AAWW,AB,ABBV,...,YUMC,YY,Z,ZBH,ZBRA,ZD,ZEN,ZION,ZTS,ZWS
Ticker,,,,,,,,,,,,,,,,,,,,,
A,0.004302,0.004624,0.003335,0.002292,0.002240,0.001558,0.002057,0.003055,0.002805,0.002142,...,0.002127,0.001849,0.002147,0.002376,0.004292,0.001901,0.001888,0.002286,0.001960,0.003196
AA,0.004624,0.035389,0.012813,0.001398,0.008947,0.002441,0.011458,0.011509,0.010156,0.006601,...,0.004102,0.000853,0.009487,0.007739,0.007634,0.004690,0.005006,0.012673,0.002543,0.010545
AAL,0.003335,0.012813,0.016325,0.002670,0.004528,-0.001055,0.006681,0.003653,0.006533,0.003294,...,0.002859,0.005509,0.005668,0.004139,0.006189,0.004720,0.004432,0.008674,0.001677,0.007185
AAON,0.002292,0.001398,0.002670,0.005714,0.001630,0.002205,0.001847,0.002249,0.002037,0.000928,...,0.001273,0.001327,0.003277,0.001686,0.003447,0.001122,0.002044,0.001614,0.002051,0.001757
AAP,0.002240,0.008947,0.004528,0.001630,0.010167,0.001393,0.004376,0.004176,0.004999,0.002945,...,0.001409,0.000077,0.004841,0.003339,0.004350,0.002739,0.002914,0.004717,0.002904,0.003160
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZD,0.001901,0.004690,0.004720,0.001122,0.002739,-0.003665,0.003598,0.000891,0.002858,0.001090,...,0.001916,0.000264,0.005429,0.003132,0.003497,0.006257,0.003257,0.003216,0.000972,0.004113
ZEN,0.001888,0.005006,0.004432,0.002044,0.002914,0.001534,0.003439,0.003100,0.003461,0.002252,...,0.001614,0.003695,0.006104,0.003618,0.004501,0.003257,0.008993,0.003649,0.001241,0.005496
ZION,0.002286,0.012673,0.008674,0.001614,0.004717,0.002296,0.005342,0.004257,0.005838,0.002905,...,0.002663,0.003931,0.005152,0.003892,0.005338,0.003216,0.003649,0.008934,0.001065,0.005357


Here is our regression to get the beta and alpha on each stock.

In [47]:
total_data = pd.DataFrame()
for column in pt_ret:
    model = smf.ols(f'{column} ~ VOO', data=pt_ret).fit()
    beta = model.params['VOO']
    alpha = model.params['Intercept']
    stderrA = model.bse['Intercept']
    stderrB = model.bse['VOO']
    rsquared = model.rsquared
    stats_data = pd.DataFrame({'Beta': [beta], 'Alpha': [alpha], 'StdErrA': [stderrA], 'StdErrB': [stderrB], 'R-squared':[rsquared]}, index=[column])
    total_data = pd.concat([total_data, stats_data])

total_data.index = total_data.index.rename('Ticker')
capm = ((risk_free_rate + total_data['Beta']) * equity_risk_premium)
adjCapm = ((risk_free_rate + total_data['Beta'] + total_data['Alpha']) * equity_risk_premium)
sharpe_ratio_arth = ((risk_free_rate + capm)/np.sqrt(arth_pt_var))
sharpe_ratio_log = ((risk_free_rate + capm)/np.sqrt(log_pt_var))
sortino_ratio_arth = ((risk_free_rate + capm)/np.sqrt(arth_pt_dvar))
sortino_ratio_log = ((risk_free_rate + capm)/np.sqrt(log_pt_dvar))
total_data['CAPM'] = capm
total_data['AdjCAPM'] = adjCapm
total_data['Sharpe Ratio (Arth)'] = sharpe_ratio_arth
total_data['Sharpe Ratio (Log)'] = sharpe_ratio_log
total_data['Sortino Ratio (Arth)'] = sortino_ratio_arth
total_data['Sortino Ratio (Log)'] = sortino_ratio_log

total_data = total_data.join(stock_data[['Market Value', 'Common Shares', 'Purchase Price', 'Recent Closing Price']])

total_data

,Beta,Alpha,StdErrA,StdErrB,R-squared,CAPM,AdjCAPM,Sharpe Ratio (Arth),Sharpe Ratio (Log),Sortino Ratio (Arth),Sortino Ratio (Log),Market Value,Common Shares,Purchase Price,Recent Closing Price
Ticker,,,,,,,,,,,,,,,
A,1.043639,0.005625,0.006146,0.132468,0.508476,0.057539,0.057837,0.434896,0.438112,0.724044,0.675495,38651.1,324.0,43.6,130.6
AA,2.261134,0.003817,0.019419,0.418547,0.327244,0.122066,0.122268,0.265421,0.251763,0.401574,0.293522,7085.5,182.9,28.1,39.4
AAL,1.573867,-0.025575,0.013198,0.284457,0.337841,0.085641,0.084285,0.301429,0.288381,0.445487,0.371797,8578.0,507.3,46.7,13.2
AAON,0.657340,0.002848,0.009163,0.197493,0.155861,0.037065,0.037216,0.303649,0.301949,0.515550,0.471324,2950.8,52.7,33.1,55.6
AAP,1.262513,-0.006624,0.010963,0.236291,0.322402,0.069139,0.068788,0.319623,0.318180,0.506903,0.437358,10211.7,73.7,169.1,169.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZD,0.969744,-0.006326,0.009085,0.195808,0.290171,0.053622,0.053287,0.339654,0.348975,0.600219,0.548554,3501.6,47.4,81.8,74.2
ZEN,1.014720,0.018553,0.011195,0.241287,0.227657,0.056006,0.056989,0.294683,0.298336,0.516968,0.457500,9445.1,96.7,21.2,76.5
ZION,1.279064,-0.004125,0.009315,0.200776,0.403488,0.070016,0.069798,0.355723,0.342109,0.466174,0.390166,7806.5,203.1,43.0,51.9


Now Here is our Stats Screen where 
1. Purchase Price has to be greater than the Recent Closing Price  
2. Take the 100 Largest Soritno Stocks with positive alpha only and pair them to the least correlated stock.
3. Take the 100 Largest Market Cap Stocks and pair them with the least correlated stocks in the data. 


In [48]:
# Here is the Correlation of Returns
correlation_of_arth_returns = pt_ret.corr() # Correlation Matrix of the arthmatic returns
correlation_of_log_returns = pt_log.corr() # Correlation Matrix of the log returns


# Purchase Price has to be less than recent closing price
data_screen = total_data[total_data['Purchase Price'] < total_data['Recent Closing Price']]
data_screen = data_screen[(data_screen['Alpha'] > 0) & (data_screen['Beta'] < 3)]

# List the larget sortino ratio stocks by using nthlarget
largest_arth_sortino = data_screen.nlargest(100, 'Sortino Ratio (Arth)')
largest_log_sortino = data_screen.nlargest(100, 'Sortino Ratio (Log)')


largest_sortino = largest_log_sortino.head(100)

# Create a list of the tickers from largest_sortino
tickers_list_of_largest_sortino = largest_sortino.index.tolist()


# Create a Pandas Dataframe that pairs tickers_list_of_largest_sortino with the lowest correlated stocks from the correlation_of_log_returns. The First column is the tickers from tickers_list_of_largest_sortino. The Second column is the tickers that have the lowest correlation to tickers_list_of_largest_sortino. The Third column is the correlation between the first and second column
lowest_correlation = pd.DataFrame()
for ticker in tickers_list_of_largest_sortino:
    lowest_correlation = pd.concat([lowest_correlation,
     pd.DataFrame({'Ticker': [ticker], 'Paired Tickers': [correlation_of_log_returns[ticker].nsmallest(2).index[1]],
     'Correlation': [correlation_of_log_returns[ticker].nsmallest(2)[1]]})])
lowest_correlation = lowest_correlation.set_index('Ticker')
average_correlation = lowest_correlation['Correlation'].mean()


lowest_correlation.head(100)


,Paired Tickers,Correlation
Ticker,,
IVT,DVN,-0.889210
LIN,CTRA,-0.161139
MSFT,CTRA,-0.186362
NOW,CALM,-0.213157
ORCL,MKC,-0.195300
JBHT,VTRS,-0.118894
UNH,CLDX,-0.153939
ABT,KDNY,-0.145779
ROP,VIRT,-0.172423


Here we are showing what the least correlated stocks are to the S&P 500. 

In [49]:
least_correlated = correlation_of_log_returns['VOO'].nsmallest(100)
least_correlated.head(10)

Ticker
VIRT   -0.153171
CSR    -0.150530
CALM   -0.104158
CTRA   -0.094396
WDFC   -0.090049
NVAX   -0.056353
QDEL   -0.050847
DAC    -0.040316
KDNY   -0.027968
COOP   -0.009143
Name: VOO, dtype: float64

Here we are taking the top 100 Market Cap Stocks in the NASDAQ & NSYE. 

In [50]:
# Top 50 Largest Market Cap Stocks
largest_market_cap = stock_data.nlargest(100, 'Market Value')
largest_market_cap = largest_market_cap[largest_market_cap['Purchase Price'] < largest_market_cap['Recent Closing Price']]
# Remove BRK.B from the list of largest_market_cap
largest_market_cap = largest_market_cap[largest_market_cap.index != 'BRK.B']

ticker_list_of_largest_market_cap = largest_market_cap.index.tolist()
largest_market_cap.head(10)

,Name,Stock Exchange,RBICS Economy,Recent Closing Price,Market Value,Purchase Price,Common Shares
Ticker,,,,,,,
AAPL,Apple Inc.,NASDAQ,Technology,142.4,2288636.0,28.2,21344.7
MSFT,Microsoft Corporation,NASDAQ,Technology,237.5,1771473.0,68.9,7708.0
GOOGL,Alphabet Inc. Class A,NASDAQ,Technology,100.0,1309220.0,39.6,13825.9
AMZN,"Amazon.com, Inc.",NASDAQ,Consumer Non-Cyclicals,113.8,1159242.0,37.5,9540.0
TSLA,Tesla Inc,NASDAQ,Consumer Cyclicals,219.4,687326.7,14.2,2423.4
UNH,UnitedHealth Group Incorporated,NYSE,Healthcare,521.9,488157.5,160.0,952.0
JNJ,Johnson & Johnson,NYSE,Healthcare,166.6,437995.1,115.2,2706.5
XOM,Exxon Mobil Corporation,NYSE,Energy,100.6,419347.6,90.3,4148.0
V,Visa Inc. Class A,NYSE,Finance,185.3,383250.9,82.7,2340.6


Now We are Getting the Largest Market Cap Stocks and pairing it with the least negative in our overall initial screen. 

In [52]:
lowest_correlation = pd.DataFrame()
for ticker in ticker_list_of_largest_market_cap:
    lowest_correlation = pd.concat([lowest_correlation,
     pd.DataFrame({'Ticker': [ticker], 'Paired Tickers': [correlation_of_log_returns[ticker].nsmallest(2).index[1]],
     'Correlation': [correlation_of_log_returns[ticker].nsmallest(2)[1]]})])
lowest_correlation = lowest_correlation.set_index('Ticker')
average_correlation = lowest_correlation['Correlation'].mean()
lowest_correlation.head(10)


,Paired Tickers,Correlation
Ticker,,
AAPL,CPE,-0.454709
MSFT,CTRA,-0.186362
GOOGL,CALM,-0.245629
AMZN,COOP,-0.295839
TSLA,LXU,-0.276354
UNH,CLDX,-0.153939
JNJ,KDNY,-0.241086
XOM,NVAX,-0.209198
V,QDEL,-0.287185
